# 2. Gadget decomposition

**GOAL:** for a given polynomial $\boldsymbol{a}$ and an encryption $\textsf{RLWE}(m)$ (or something similar to it), find $\textsf{RLWE}(\boldsymbol{a}\cdot \boldsymbol{m})$, with small noise.

Gadget decomposition and $RLWE'$ technique allows us to multiply ciphertext to a *large* constant with *small* noise increment.

RLWE' is usually used for the key switching. We make an example of key switching from s1 to s2 here.

In [1]:
# Functions from previous lecturenote
import torch
import math

stddev = 3.2
logQ = 27

N = 2**10
Q = 2**logQ


def keygen(dim):
    return torch.randint(2, size = (dim,))

def errgen(stddev):
    e = torch.round(stddev*torch.randn(1))
    e = e.squeeze()
    return e.to(torch.int)

def errgen(stddev, N):
    e = torch.round(stddev*torch.randn(N))
    e = e.squeeze()
    return e.to(torch.int)

def uniform(dim, modulus):
    return torch.randint(modulus, size = (dim,))

def polymult(a, b, dim, modulus):
    res = torch.zeros(dim).to(torch.int)
    for i in range(dim):
        for j in range(dim):
            if i >= j:
                res[i] += a[j]*b[i-j]
                res[i] %= modulus
            else:
                res[i] -= a[j]*b[i-j] # Q - x mod Q = -x
                res[i] %= modulus

    res %= modulus
    return res

root_powers = torch.arange(N//2).to(torch.complex128)
root_powers = torch.exp((1j*math.pi/N)*root_powers)

root_powers_inv = torch.arange(0,-N//2,-1).to(torch.complex128)
root_powers_inv = torch.exp((1j*math.pi/N)*root_powers_inv)

def negacyclic_fft(a, N, Q):
    acomplex = a.to(torch.complex128)

    a_precomp = (acomplex[...,:N//2] + 1j * acomplex[..., N//2:]) * root_powers

    return torch.fft.fft(a_precomp)

def negacyclic_ifft(A, N, Q):
    b = torch.fft.ifft(A)
    b *= root_powers_inv

    a = torch.cat((b.real, b.imag), dim=-1)

    aint = a.to(torch.int32)
    # only when Q is a power-of-two
    aint &= Q-1

    return aint

In [2]:
# generate two keys
s1 = keygen(N)
s2 = keygen(N)

s1fft = negacyclic_fft(s1, N, Q)
s2fft = negacyclic_fft(s2, N, Q)

In [3]:
# make an RLWE encryption of message
def encrypt_to_fft(m, sfft):
    ct = torch.stack([errgen(stddev, N), uniform(N, Q)])
    ctfft = negacyclic_fft(ct, N, Q)

    ctfft[0] += -ctfft[1]*sfft + negacyclic_fft(m, N, Q)

    return ctfft

def decrypt_from_fft(ctfft, sfft):
    return negacyclic_ifft(ctfft[0] + ctfft[1]*sfft, N, Q)

In [4]:
m = torch.zeros((N), dtype=torch.int32)
m[0] = 1000000

m

tensor([1000000,       0,       0,  ...,       0,       0,       0],
       dtype=torch.int32)

In [5]:
ctfft = encrypt_to_fft(m, s1fft)
mdec = decrypt_from_fft(ctfft, s1fft)
mdec

tensor([  1000003,         2, 134217726,  ...,         5,         6,
                1], dtype=torch.int32)

We cannot decrypt mdec with s2, as the secret key is different. It should look like a random value.

In [6]:
mdec_wrong = decrypt_from_fft(ctfft, s2fft)
mdec_wrong

tensor([ 13737172, 127554412,  96457619,  ...,  49456183,  19749892,
         37101897], dtype=torch.int32)

We want to transform ct as a ciphertext of the same message but with different key s2 *without decryption*.

## 2.1. Gadget decomposition

We define *gadget decomposition* $h$ corresponding to gadget vector $\vec{g} = (g_0, g_1, \dots, g_{d-1})$ as follows.
$$
h: \mathbb{Z} \longmapsto \mathbb{Z}^d
$$
$$
||h(a)|| < B, \left< h(a), \vec{g}\right> = a,
$$
where B is a upper bound.

Also, we can naturally extend it to $\mathcal{R}$ and $\mathbb{Z}^n$.

For example, a number $77 = 0\text{b}01001101$ can be decomposed to $(0,1,0,0,1,1,0,1)$ when $\vec{g} = (2^7, 2^6, \dots, 1)$ and $B=2$

Otherwise, it can also be decomposed to $(1,0,3,1)$ when $\vec{g} = (4^3, 4^2, 4, 1)$, and $B=4$ here.

We also can do a *approximate gadget decomposition* where $\left< h(a), \vec{g}\right> \approx a$, i.e., $\left< h(a), \vec{g}\right>$ is not exactly the same but similar to $a$.

We first set d and B satisfying $B^{d} \le Q \le B^{d+1}$

In [7]:
# we will use B-ary decomposition, i.e., cut digits by logB bits
d = 3
logB = 7

In [8]:
decomp_shift = logQ - logB*torch.arange(1,d+1).unsqueeze(dim = 1)
decomp_shift

tensor([[20],
        [13],
        [ 6]])

In [9]:
# ((1<<logB) -1) is 0b1111111 (len of 1 is logB)
decomp_mask = ((1<<logB) -1) << decomp_shift
decomp_mask

tensor([[133169152],
        [  1040384],
        [     8128]])

In [10]:
gvector = 1<<decomp_shift
gvector

tensor([[1048576],
        [   8192],
        [     64]])

In [11]:
def decompose(a):
    da = a.unsqueeze(dim = a.dim()-1)
    
    newdim = list(da.size())
    newdim[-2] = d
    
    da = da.expand(newdim)

    return (da & decomp_mask) >> decomp_shift

In [12]:
a = uniform(N, Q)
a

tensor([ 65261118, 125653224,   7230529,  ...,  82140274,  67572063,
         19397401])

In [13]:
da = decompose(a)

In [14]:
#composition is inner product, see it is similar to a
torch.sum(da * gvector, dim = 0)

tensor([ 65261056, 125653184,   7230528,  ...,  82140224,  67572032,
         19397376])

We can extend it to a ciphertext too

In [15]:
ctfft = encrypt_to_fft(m, s1fft)
ct = negacyclic_ifft(ctfft, N, Q)
ct

tensor([[  4486380,   5226831,   3046802,  ...,  86454026,  62226672,
          61140349],
        [ 93497099,   7935531, 116776838,  ...,  51850894,  41694239,
          50890776]], dtype=torch.int32)

In [16]:
torch.sum(decompose(ct) * gvector, dim = 1)

tensor([[  4486336,   5226816,   3046784,  ...,  86454016,  62226624,
          61140288],
        [ 93497088,   7935488, 116776832,  ...,  51850880,  41694208,
          50890752]])